<a href="https://colab.research.google.com/github/nahumsa/DM-Reconstruction/blob/Tensorflow_test/Trace%20Distance%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.2.0-rc3


# Tests

In [0]:
#Creating pauli matrices
sigma_0_np = np.array([[1.,0.],
                      [0.,1.]], dtype=np.complex64)

sigma_1_np = np.array([[0.,1.],
                       [1.,0.]], dtype=np.complex64)

sigma_2_np = np.array([[0.,1.j],
                       [-1.j,0.]], dtype=np.complex64)

sigma_3_np = np.array([[1.,0.],
                       [0.,-1.]], dtype=np.complex64)

#Converting to tensors
sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

In [79]:
from itertools import product
name_basis_1 = ['I', 'X', 'Y', 'Z']
basis_1 = [sigma_0, sigma_1,sigma_2,sigma_3]
name_basis_2 = []
basis_2 = []
for (name_1, meas_1),(name_2,meas_2) in product(zip(name_basis_1, basis_1),zip(name_basis_1, basis_1)):
  if name_1 == 'I' and name_2 == 'I':
    pass
  else:
    basis_2.append(tf.tensordot(meas_1,meas_2,axes=0))
    name_basis_2.append(name_1 + name_2)

basis_2_tf = tf.Variable(basis_2)
print(name_basis_2)
print(basis_2_tf.shape)

['IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI', 'ZX', 'ZY', 'ZZ']
(15, 2, 2, 2, 2)


In [95]:
#meas = [np.random.rand(15),np.random.rand(15),np.random.rand(15),np.random.rand(15)]
#measurement = np.array(meas, dtype=np.complex64)
meas_1 = [np.random.rand(15)]
measurement = np.array(meas_1, dtype=np.complex64)
measurement_tensor = tf.Variable(measurement, tf.complex64)
#print(measurement_tensor)

<tf.Variable 'Variable:0' shape=(1, 15) dtype=complex64, numpy=
array([[0.29690507+0.j, 0.7638097 +0.j, 0.9178417 +0.j, 0.41568556+0.j,
        0.596676  +0.j, 0.22739619+0.j, 0.8641635 +0.j, 0.71199936+0.j,
        0.96396905+0.j, 0.244707  +0.j, 0.20325921+0.j, 0.46136403+0.j,
        0.47117528+0.j, 0.931769  +0.j, 0.5679348 +0.j]], dtype=complex64)>


In [89]:
ones_II = tf.ones((measurement_tensor.shape[0],1), dtype=tf.dtypes.complex64)
II = tf.Variable([tf.tensordot(sigma_0 , sigma_0, axes=0)])
density_matrix = 0.25*(tf.tensordot(ones_II , II ,axes=1) + tf.tensordot(measurement_tensor,basis_2_tf,axes=1))
print(density_matrix.shape)

(1, 2, 2, 2, 2)


In [94]:
mean_density_matrix = tf.reduce_sum(density_matrix, axis=0)
print(tf.linalg.trace(tf.linalg.trace(mean_density_matrix)))

tf.Tensor((1+0j), shape=(), dtype=complex64)


In [60]:
tf.linalg.eigvalsh(density_matrix)

<tf.Tensor: shape=(2, 2, 2), dtype=complex64, numpy=
array([[[ 0.12169091+0.j,  0.5714514 +0.j],
        [-0.00929919+0.j,  0.24463826+0.j]],

       [[-0.09903219+0.j,  0.33437124+0.j],
        [ 0.06098602+0.j,  0.24587153+0.j]]], dtype=complex64)>

# Methods

## One qubit

In [0]:
def create_density_mat(measurements: np.array) -> tf.Tensor:
  
  #Creating the basis
  sigma_0_np = np.array([[1.,0.],
                        [0.,1.]], dtype=np.complex64)

  sigma_1_np = np.array([[0.,1.],
                         [1.,0.]], dtype=np.complex64)

  sigma_2_np = np.array([[0.,1.j],
                         [-1.j,0.]], dtype=np.complex64)

  sigma_3_np = np.array([[1.,0.],
                        [0.,-1.]], dtype=np.complex64)

  #Converting to tensors
  sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

  sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

  sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

  sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

  basis = tf.Variable([sigma_1,sigma_2,sigma_3])
  
  measurement = np.array(measurements, dtype=np.complex64)
  measurement_tensor = tf.Variable(measurement, tf.complex64)

  density_matrix = 0.5*(sigma_0 + tf.tensordot(measurement_tensor,basis,axes=1))
  return density_matrix


In [0]:
def trace_dist(A,B):
  dif = A - B
  dif = tf.transpose(dif, conjugate=True, perm=[0,2,1]) * dif  
  vals = tf.linalg.eigvalsh(dif)
  return tf.math.real(0.5*tf.reduce_sum(tf.math.sqrt(tf.math.abs(vals)),axis=-1))

In [0]:
def trace_loss(y_true,y_pred):
  d_y_true = create_density_mat(y_true)  
  d_y_pred = create_density_mat(y_pred)  
  return tf.reduce_mean(trace_dist(d_y_pred,d_y_true))

In [0]:
y_1 = [[.5,.5,0],[.2,0.3,.3]]
y_2 = [[1,0,0],[.2,.3,.3]]
trace_loss(y_true=y_1, y_pred=y_2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.17677669>

## Two qubits

In [0]:
from itertools import product
#Creating pauli matrices
sigma_0_np = np.array([[1.,0.],
                      [0.,1.]], dtype=np.complex64)

sigma_1_np = np.array([[0.,1.],
                       [1.,0.]], dtype=np.complex64)

sigma_2_np = np.array([[0.,1.j],
                       [-1.j,0.]], dtype=np.complex64)

sigma_3_np = np.array([[1.,0.],
                       [0.,-1.]], dtype=np.complex64)

#Converting to tensors
sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

def create_2qubit_density_mat(measurements: tf.Variable) -> tf.Variable:
  
  name_basis_1 = ['I', 'X', 'Y', 'Z']
  basis_1 = [sigma_0, sigma_1,sigma_2,sigma_3]
  name_basis_2 = []
  basis_2 = []
  for (name_1, meas_1),(name_2,meas_2) in product(zip(name_basis_1, basis_1),zip(name_basis_1, basis_1)):
    if name_1 == 'I' and name_2 == 'I':
      pass
    else:
      basis_2.append(tf.tensordot(meas_1,meas_2,axes=0))
      name_basis_2.append(name_1 + name_2)
  basis_2_tf = tf.Variable(basis_2)
  ones_II = tf.ones((measurement_tensor.shape[0],1), dtype=tf.dtypes.complex64)
  II = tf.Variable([tf.tensordot(sigma_0 , sigma_0, axes=0)])
  density_matrix = 0.25*(tf.tensordot(ones_II , II ,axes=1) + tf.tensordot(measurement_tensor,basis_2_tf,axes=1))
  return density_matrix

In [98]:
create_2qubit_density_mat(meas_1)

<tf.Tensor: shape=(1, 2, 2, 2, 2), dtype=complex64, numpy=
array([[[[[ 0.7367851 +0.j        ,  0.19202009+0.42389467j],
          [ 0.19202009-0.42389467j, -0.0061031 +0.j        ]],

         [[ 0.31996226+0.22881465j,  0.08799225+0.2978413j ],
          [ 0.21034575+0.18414322j, -0.11211949+0.12718503j]]],


        [[[ 0.31996226-0.22881465j,  0.21034575-0.18414322j],
          [ 0.08799225-0.2978413j , -0.11211949-0.12718503j]],

         [[ 0.22213571+0.j        , -0.04356755-0.04198983j],
          [-0.04356755+0.04198983j,  0.04718228+0.j        ]]]]],
      dtype=complex64)>